<a href="https://colab.research.google.com/github/cimbelli/Surface-Heat-Islands/blob/main/temperatura_max_mensile_capoluoghi-regione_1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stima delle temperatura massima mensile nei capoluoghi di regione

In [6]:
import os
import ee
import pandas as pd
import json
import geopandas as gpd
import requests
#from urllib.request import urlopen
#ee.Authenticate()
ee.Initialize()
!pip install rasterstats
!pip install wget
import wget
from rasterstats import zonal_stats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=2d82dd319c7ae3533c001c1570bc239eae20c819a5a04055ba817d1d813a76e9
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [7]:
anno = 2023
mesi = [6,7,8,9]

In [8]:
import folium
from folium import plugins

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
def crea_mappa(nomecom,periodo):
  m = folium.Map(location=(centro.y[0],centro.x[0] ), tiles="cartodb positron", zoom_start=12)
  folium.Choropleth(
      geo_data=df_sez11,
      name="Media per sezione delle temperature massime mensili",
      data=sdf1,
      columns=["SEZ", "m202308"],
      key_on="feature.properties.SEZ",
      fill_color= 'YlOrRd', #YlGn
      fill_opacity=0.7,
      line_opacity=0.2,
      legend_name="Unemployment Rate (%)",
  ).add_to(m)

  folium.LayerControl().add_to(m)
  m.save(comune + '_' + str(anno) + mese1 + '.html')

In [9]:
def scarica(lyr, name, aoi):
    url = lyr.getDownloadUrl({
        'name': name[:-4],
        'bands': ['ST'],
        'region': aoi,
        'scale': 100,
        'format': 'GEO_TIFF'
    })
    response = requests.get(url)
    with open(name, 'wb') as fd:
        fd.write(response.content)
    return None

In [10]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [76]:
# carica lo shape dei capoluoghi di regione
nomecap = 'comregll3.geojson'
if not(os.path.isfile(nomecap)):
  wget.download('https://github.com/cimbelli/Surface-Heat-Islands/raw/5006ef2e37091eedf421f65ed9cbf80c15ccbdfd/' + nomecap)
g = open(nomecap)
cc = json.load(g)

gdf = gpd.read_file('comregll3.geojson')
centro = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs)
#gdf["lon"] = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs) #gdf["geometry"].to_crs(4326).centroid.x
#gdf["lat"] = gdf["geometry"].centroid.y
gdf["lon"] = centro.x
gdf["lat"] = centro.y

In [79]:
try:
  del sdf
except:
  pass
for i in range(len(cc['features'])):

    reg = cc['features'][i]['properties']['REG']
    reg1 = ('0' if reg < 10 else '') + str(reg)
    id1 = cc['features'][i]['properties']['PRO_COM']

    coords = cc['features'][i]['geometry']['coordinates']
    fc = ee.FeatureCollection(cc['features'])
    comune = cc['features'][i]['properties']['COMUNE']

    nomefilesez = 'R' + reg1 + '_11geo.geojson'
    if not(os.path.isfile(nomefilesez)):
      urlfilesez = 'https://github.com/cimbelli/Surface-Heat-Islands/raw/606eb4915031627b8b1f38c3d29c5e4cfef7233f/sez11/' + nomefilesez
      wget.download(urlfilesez)
    g1 = open(nomefilesez)
    df_sez11 = gpd.read_file(g1)
    df_sez11['PRO_COM'] = df_sez11['PRO_COM'].astype(int)
    df_sez11['SEZ'] = df_sez11['SEZ'].astype(int)

    lat = gdf[gdf['PRO_COM']== id1]['lat'].values[0]
    lon = gdf[gdf['PRO_COM']== id1]['lon'].values[0]

    minx = gdf[gdf['PRO_COM']== id1].geometry.bounds.minx.values[0]
    maxx = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxx.values[0]
    miny = gdf[gdf['PRO_COM']== id1].geometry.bounds.miny.values[0]
    maxy = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxy.values[0]

    region = ee.Geometry.BBox(minx, miny, maxx, maxy)

    for mese in mesi:
        nome_img = 'ST_' + comune + '_'+ str(anno) + '0' + str(mese) + '.tif'
        mese2 = ('0' + str(mese+1)) if mese < 9 else str(mese+1)

        if not(os.path.isfile(nome_img)):

            L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(anno) + '-0' + str(mese) +'-01', str(anno) + '-' + mese2 +'-01')
            L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(str(anno) + '-0' + str(mese) +'-01', str(anno) + '-' + mese2 +'-01')
            bands = ['ST_B10', 'QA_PIXEL']
            bandName = ['ST', 'QA_PIXEL']
            filtered_L9 = L9.filter(ee.Filter.lt('CLOUD_COVER', 20)).select(bands, bandName)#.map(cloudMask)
            filtered_L8 = L8.filter(ee.Filter.lt('CLOUD_COVER', 20)).select(bands, bandName)#.map(cloudMask)
            LandsatColl = filtered_L8.merge(filtered_L9)
            LandsatColl = LandsatColl.select('ST').filterBounds(region)

            lst = LandsatColl.max().clip(fc)
            thermal = lst.select('ST.*').multiply(0.00341802).add(149.0).subtract(273.15)


            # zonal stats per sezioni
            print('scarico immagine', comune, anno, "mese",mese, "n° acquisizioni", LandsatColl.size().getInfo())

            scarica(thermal, nome_img, region)
        else:
            print(nome_img, 'già presente')

 # se è il primo mese usa il df delle sezioni originario altrimenti quello con precedenti stats
        try:
            df_o = sdf
            #print("sdf esistente")
        except:
            df_o = df_sez11
            #stats = zonal_stats(df_o, nome_img, stats="mean")

            #print("sdf non esistente")
        stats = zonal_stats(df_o, nome_img, stats="mean")
        sdf = pd.DataFrame(stats)
        sdf.rename(columns={'mean': 'm' + str(anno) + '0' + str(mese) }, inplace=True)
        sdf = pd.concat([df_o, sdf], axis=1)
        stats.clear()

        #print('regione', reg, comune, 'anno', anno, 'mese', mese)#, center)#, aoi)
    sdf1 = sdf.drop('geometry', axis=1) # sdf.drop('geometry', inplace=True, axis=1)
    sdf1.to_excel(comune + '_' + 'stats.xlsx', index=False)
    sdf1.to_csv(comune + '_' + 'stats.csv', index=False)

    #del sdf
    print("create statistiche mensili per", comune, "estate", anno)

    # Crea la mappa per il comune e il mese di agosto
    m = folium.Map(location=(centro.y[0],centro.x[0] ), tiles="cartodb positron", zoom_start=12)
    folium.Choropleth(
        geo_data=df_sez11,
        name ="Media per sezione delle temperature massime mensili - " + comune + '-' + 'Agosto',
        data=sdf1,
        columns=["SEZ", "m202308"],
        key_on="feature.properties.SEZ",
        fill_color= 'YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Unemployment Rate (%)",
    ).add_to(m)

    folium.LayerControl().add_to(m)
    m.save(comune + '_' + str(anno) + '08' + '.html')
    print("Creata mappa interattiva per", comune, "agosto", anno)

    break


ST_Milano_202306.tif già presente
ST_Milano_202307.tif già presente
ST_Milano_202308.tif già presente
ST_Milano_202309.tif già presente
create statistiche mensili per Milano estate 2023
Creata mappa interattiva per Milano agosto 2023


In [35]:
sdf1

,PRO_COM,SEZ2011,SEZ,m202306,m202307,m202308,m202309
0,15146,151460005984,5984,46.371600,45.479497,52.045513,35.868024
1,15146,151460001209,1209,50.862878,49.398256,54.774802,38.211077
2,15146,151460001062,1062,51.254241,49.432437,55.613926,38.466574
3,15146,151460005439,5439,48.877008,47.403842,52.881219,37.313847
4,15146,151460000748,748,48.545460,46.115248,51.495212,37.255740
...,...,...,...,...,...,...,...
6080,15146,151460002316,2316,46.867213,43.691872,50.104078,35.013519
6081,15146,151460003011,3011,46.003023,42.044386,48.339240,34.647791
6082,15146,151460003017,3017,41.976596,38.179745,46.004163,31.778933
6083,15146,151460003028,3028,41.103454,39.915937,46.814478,33.235661


In [72]:
# Crea la mappa per il comune e il mese
m = folium.Map(location=(centro.y[0],centro.x[0] ), tiles="cartodb positron", zoom_start=12)
folium.Choropleth(
    geo_data=df_sez11,
    name="Media per sezione delle temperature massime mensili",
    data=sdf1,
    columns=["SEZ", "m202308"],
    key_on="feature.properties.SEZ",
    fill_color= 'YlOrRd', #YlGn
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)
m.save(comune + '_' + str(anno) + mese1 + '.html')
m